# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 12:36PM,247161,16,8561892,Sartorius Bioprocess Solutions,Released
1,Sep 23 2022 12:36PM,247161,16,8563948,Sartorius Bioprocess Solutions,Released
2,Sep 23 2022 12:36PM,247161,16,8563988,Sartorius Bioprocess Solutions,Released
3,Sep 23 2022 12:36PM,247161,16,8563987,Sartorius Bioprocess Solutions,Released
4,Sep 23 2022 12:30PM,247160,12,HH-37088.1,Hush Hush,Executing
5,Sep 23 2022 12:30PM,247160,12,HH-37089.1,Hush Hush,Executing
6,Sep 23 2022 12:30PM,247160,12,HH-37091.1,Hush Hush,Executing
7,Sep 23 2022 12:24PM,247157,10,0086003230,ISDIN Corporation,Released
8,Sep 23 2022 12:24PM,247157,10,0086003277,ISDIN Corporation,Released
9,Sep 23 2022 12:24PM,247157,10,0086003287,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,247156,Released,2
29,247157,Released,9
30,247159,Released,1
31,247160,Executing,3
32,247161,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247156,NaN,2.0,2.0
247157,NaN,NaN,9.0
247159,NaN,NaN,1.0
247160,NaN,3.0,NaN
247161,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0.0,1.0,0.0
247000,0.0,1.0,0.0
247037,0.0,23.0,11.0
247083,5.0,1.0,4.0
247100,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0,1,0
247000,0,1,0
247037,0,23,11
247083,5,1,4
247100,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,0,1,0
1,247000,0,1,0
2,247037,0,23,11
3,247083,5,1,4
4,247100,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,,1,
1,247000,,1,
2,247037,,23,11
3,247083,5,1,4
4,247100,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions
4,Sep 23 2022 12:30PM,247160,12,Hush Hush
7,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation
16,Sep 23 2022 12:22PM,247159,19,ACG North America LLC
17,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc."
51,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC"
55,Sep 23 2022 12:00PM,247152,10,Emerginnova
57,Sep 23 2022 11:49AM,247151,10,ISDIN Corporation
80,Sep 23 2022 11:44AM,247150,10,ISDIN Corporation
93,Sep 23 2022 10:58AM,247117,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,,,4
1,Sep 23 2022 12:30PM,247160,12,Hush Hush,,3,
2,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,,,9
3,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,,,1
4,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",,23,11
5,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC",,2,2
6,Sep 23 2022 12:00PM,247152,10,Emerginnova,,,2
7,Sep 23 2022 11:49AM,247151,10,ISDIN Corporation,,,23
8,Sep 23 2022 11:44AM,247150,10,ISDIN Corporation,,,13
9,Sep 23 2022 10:58AM,247117,15,"Alliance Pharma, Inc.",,2,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,4,,
1,Sep 23 2022 12:30PM,247160,12,Hush Hush,,3,
2,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,9,,
3,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,1,,
4,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",11,23,
5,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC",2,2,
6,Sep 23 2022 12:00PM,247152,10,Emerginnova,2,,
7,Sep 23 2022 11:49AM,247151,10,ISDIN Corporation,23,,
8,Sep 23 2022 11:44AM,247150,10,ISDIN Corporation,13,,
9,Sep 23 2022 10:58AM,247117,15,"Alliance Pharma, Inc.",15,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,4,,
1,Sep 23 2022 12:30PM,247160,12,Hush Hush,,3,
2,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,9,,
3,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,1,,
4,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",11,23,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,4.0,NaN,NaN
1,Sep 23 2022 12:30PM,247160,12,Hush Hush,NaN,3.0,NaN
2,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,9.0,NaN,NaN
3,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,1.0,NaN,NaN
4,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",11.0,23.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,4.0,0.0,0.0
1,Sep 23 2022 12:30PM,247160,12,Hush Hush,0.0,3.0,0.0
2,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,9.0,0.0,0.0
3,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,1.0,0.0,0.0
4,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",11.0,23.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2471380,72.0,1.0,7.0
12,247160,0.0,3.0,0.0
15,741310,28.0,27.0,0.0
16,741414,6.0,0.0,0.0
19,988535,4.0,2.0,0.0
20,247143,6.0,0.0,0.0
21,988122,2.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2471380,72.0,1.0,7.0
1,12,247160,0.0,3.0,0.0
2,15,741310,28.0,27.0,0.0
3,16,741414,6.0,0.0,0.0
4,19,988535,4.0,2.0,0.0
5,20,247143,6.0,0.0,0.0
6,21,988122,2.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,72.0,1.0,7.0
1,12,0.0,3.0,0.0
2,15,28.0,27.0,0.0
3,16,6.0,0.0,0.0
4,19,4.0,2.0,0.0
5,20,6.0,0.0,0.0
6,21,2.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,72.0
1,12,Released,0.0
2,15,Released,28.0
3,16,Released,6.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,1.0,3.0,27.0,0.0,2.0,0.0,2.0
Released,72.0,0.0,28.0,6.0,4.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,3.0,27.0,0.0,2.0,0.0,2.0
2,Released,72.0,0.0,28.0,6.0,4.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,3.0,27.0,0.0,2.0,0.0,2.0
2,Released,72.0,0.0,28.0,6.0,4.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()